In [1]:
import pandas as pd
import numpy as np
import os, glob
from random import sample
import datetime
import re

In [2]:
#initialise an empty dataframe
paed3_data = pd.DataFrame()

#fetch files from folder
file_path ='/Users/joycemungai/ds_projects/emergency_dept_files'
paed3_file_list = glob.glob(file_path + "/*paediatrics3.xls")

#loop through the files, read data and append to a dataframe
for file in paed3_file_list:
    data = pd.read_excel(file, header=None)
    paed3_data = paed3_data.append(data,ignore_index = True)
paed3_data.head()   

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,Mater Hospital Brisbane Public,JAN2018,1,0,0,-,0,-,0,-,-,0,-
1,1,Mater Hospital Brisbane Public,JAN2018,2,1,1,100,0,0,0,0,-,0,-
2,1,Mater Hospital Brisbane Public,JAN2018,3,2,1,50,0,0,0,0,-,0,-
3,1,Mater Hospital Brisbane Public,JAN2018,4,0,0,-,0,-,0,-,-,0,-
4,1,Mater Hospital Brisbane Public,JAN2018,5,0,0,-,0,-,0,-,-,0,-


In [3]:
#assign column names  from description file
'''column	description
1	the code of the facility 
2	the name of the facility
3	the month to which the data refers
4	the triage category of the patient,  note: ‘all’ is total value
5	number of emergency department attendances
6	number of attendances where the patient was seen within clinically recommended time
7	percent of attendances where the patient was seen within clinically recommended time
8	number of attendances where the patient did not wait for treatment
9	percent of attendances where the patient did not wait for treatment
10	number of attendances where the patient was admitted to an inpatient bed
11	percent of attendances where the patient was admitted to an inpatient bed 
12	an empty column, no longer used
13	number of attendances where the patient was admitted to an inpatient bed within four hours
14	percent of attendances where the patient was admitted to an inpatient bed within four hours'''

paed3_data.columns = ['facility_code', 'facility_name', 'month', 'triage_category',
                      'num_em_attendances','num_seen_on_time', 'percent_seen_on_time',
                      'num_left_no_treatment', 'percent_left_no_treatment','num_admitted',
                      'percent_admitted','empty_col','num_admitted_in_four_hrs', 'percent_admitted_in_four_hrs'
                     ]
paed3_data.head()

,facility_code,facility_name,month,triage_category,num_em_attendances,num_seen_on_time,percent_seen_on_time,num_left_no_treatment,percent_left_no_treatment,num_admitted,percent_admitted,empty_col,num_admitted_in_four_hrs,percent_admitted_in_four_hrs
0,1,Mater Hospital Brisbane Public,JAN2018,1,0,0,-,0,-,0,-,-,0,-
1,1,Mater Hospital Brisbane Public,JAN2018,2,1,1,100,0,0,0,0,-,0,-
2,1,Mater Hospital Brisbane Public,JAN2018,3,2,1,50,0,0,0,0,-,0,-
3,1,Mater Hospital Brisbane Public,JAN2018,4,0,0,-,0,-,0,-,-,0,-
4,1,Mater Hospital Brisbane Public,JAN2018,5,0,0,-,0,-,0,-,-,0,-


In [4]:
#drop the empy column
paed3_data = paed3_data.drop(columns='empty_col')

In [5]:
paed3_data.shape

(12450, 13)

In [6]:
paed3_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12450 entries, 0 to 12449
Data columns (total 13 columns):
facility_code                   12450 non-null int64
facility_name                   12450 non-null object
month                           12450 non-null object
triage_category                 12450 non-null object
num_em_attendances              12450 non-null object
num_seen_on_time                12450 non-null object
percent_seen_on_time            12431 non-null object
num_left_no_treatment           12450 non-null object
percent_left_no_treatment       12431 non-null object
num_admitted                    12450 non-null object
percent_admitted                12431 non-null object
num_admitted_in_four_hrs        12450 non-null object
percent_admitted_in_four_hrs    12370 non-null object
dtypes: int64(1), object(12)
memory usage: 1.2+ MB


In [7]:
paed3_data.isnull().any()

facility_code                   False
facility_name                   False
month                           False
triage_category                 False
num_em_attendances              False
num_seen_on_time                False
percent_seen_on_time             True
num_left_no_treatment           False
percent_left_no_treatment        True
num_admitted                    False
percent_admitted                 True
num_admitted_in_four_hrs        False
percent_admitted_in_four_hrs     True
dtype: bool

In [8]:
paed3_data.isnull().sum()

facility_code                    0
facility_name                    0
month                            0
triage_category                  0
num_em_attendances               0
num_seen_on_time                 0
percent_seen_on_time            19
num_left_no_treatment            0
percent_left_no_treatment       19
num_admitted                     0
percent_admitted                19
num_admitted_in_four_hrs         0
percent_admitted_in_four_hrs    80
dtype: int64

In [9]:
paed3_data.sample(6)

,facility_code,facility_name,month,triage_category,num_em_attendances,num_seen_on_time,percent_seen_on_time,num_left_no_treatment,percent_left_no_treatment,num_admitted,percent_admitted,num_admitted_in_four_hrs,percent_admitted_in_four_hrs
11576,69,Hervey Bay Hospital,AUG2016,3,219,157,71.69,1,0.46,56,25.57,17,30.36
6904,45,Gatton Hospital,2019-01-01 00:00:00,5,7,5,100,2,29,0,0,0,-
6727,68,Gympie Hospital,FEB2018,2,40,30,75,0,0,4,10,1,25
12287,99999,Queensland,NOV2018,ALL,34845,25327,77.04,1130,3.24,5823,16.71,3416,58.66
5243,211,Atherton Hospital,DEC2018,ALL,258,175,79.19,20,7.75,12,4.65,7,58.33
3872,100,Stanthorpe Hospital,2019-10-01 00:00:00,3,40,37,95,1,3,8,20,8,100


In [10]:
#fill missing values for the percent columns with 0
paed3_data = paed3_data.replace('-', np.NaN)
paed3_data.fillna(0, inplace=True)

In [11]:
#clean up the month column
paed3_data['month']=paed3_data['month'].replace('AUG_2015', 'AUG2015').replace('SEP_2015', 'SEP2015').replace('OCT_2015', 'OCT2015').replace('JUL_2015', 'JUL2015')
paed3_data.month = pd.to_datetime(paed3_data.month).dt.to_period('m')

In [12]:
paed3_data.groupby('facility_name')['facility_code'].count().sample(10)

facility_name
Dirranbandi Hospital               66
Millmerran Hospital                66
Mater Hospital Brisbane Public    246
Winton Hospital                    54
Childers Hospital                  66
Toowoomba Hospital                252
Chinchilla Hospital                66
Cloncurry Hospital                 66
TOOWOOMBA HOSPITAL                 24
Moura Hospital                     48
Name: facility_code, dtype: int64

In [13]:
#remove special characters
regex = re.compile("[@_!#$%^&*()<>?/\|}{~:']")
paed3_data.facility_name = [regex.sub('',name) for name in paed3_data.facility_name] 
paed3_data.facility_name[126:131]

126    Royal Brisbane  Womens Hospital
127    Royal Brisbane  Womens Hospital
128    Royal Brisbane  Womens Hospital
129    Royal Brisbane  Womens Hospital
130    Royal Brisbane  Womens Hospital
Name: facility_name, dtype: object

In [14]:
#uppercase facility name for consistency
paed3_data.facility_name = paed3_data.facility_name.str.upper()
paed3_data.facility_name[30:35]

30    QUEEN ELIZABETH II JUBILEE HOSPITAL
31    QUEEN ELIZABETH II JUBILEE HOSPITAL
32    QUEEN ELIZABETH II JUBILEE HOSPITAL
33    QUEEN ELIZABETH II JUBILEE HOSPITAL
34    QUEEN ELIZABETH II JUBILEE HOSPITAL
Name: facility_name, dtype: object

In [15]:
#remove extra spaces from the faciliy names
paed3_data.facility_name = [re.sub(' +', ' ',name) for name in paed3_data.facility_name]
paed3_data.facility_name[126:131]  

126    ROYAL BRISBANE WOMENS HOSPITAL
127    ROYAL BRISBANE WOMENS HOSPITAL
128    ROYAL BRISBANE WOMENS HOSPITAL
129    ROYAL BRISBANE WOMENS HOSPITAL
130    ROYAL BRISBANE WOMENS HOSPITAL
Name: facility_name, dtype: object

In [16]:
paed3_data = paed3_data[paed3_data.triage_category != 'ALL']

In [17]:
paed3_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10375 entries, 0 to 12448
Data columns (total 13 columns):
facility_code                   10375 non-null int64
facility_name                   10375 non-null object
month                           10375 non-null period[M]
triage_category                 10375 non-null object
num_em_attendances              10375 non-null float64
num_seen_on_time                10375 non-null float64
percent_seen_on_time            10375 non-null object
num_left_no_treatment           10375 non-null float64
percent_left_no_treatment       10375 non-null object
num_admitted                    10375 non-null float64
percent_admitted                10375 non-null object
num_admitted_in_four_hrs        10375 non-null float64
percent_admitted_in_four_hrs    10375 non-null object
dtypes: float64(5), int64(1), object(6), period[M](1)
memory usage: 1.1+ MB


In [18]:
cols =['triage_category','percent_seen_on_time', 'percent_left_no_treatment', 
       'percent_admitted', 'percent_admitted_in_four_hrs']
paed3_data[cols] = paed3_data[cols].apply(pd.to_numeric, errors='coerce', axis=1)

In [19]:
paed3_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10375 entries, 0 to 12448
Data columns (total 13 columns):
facility_code                   10375 non-null int64
facility_name                   10375 non-null object
month                           10375 non-null period[M]
triage_category                 10375 non-null float64
num_em_attendances              10375 non-null float64
num_seen_on_time                10375 non-null float64
percent_seen_on_time            10375 non-null float64
num_left_no_treatment           10375 non-null float64
percent_left_no_treatment       10375 non-null float64
num_admitted                    10375 non-null float64
percent_admitted                10375 non-null float64
num_admitted_in_four_hrs        10375 non-null float64
percent_admitted_in_four_hrs    10375 non-null float64
dtypes: float64(10), int64(1), object(1), period[M](1)
memory usage: 1.1+ MB


In [20]:
paed3_data.describe()

,facility_code,triage_category,num_em_attendances,num_seen_on_time,percent_seen_on_time,num_left_no_treatment,percent_left_no_treatment,num_admitted,percent_admitted,num_admitted_in_four_hrs,percent_admitted_in_four_hrs
count,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000,10375.000000
mean,2358.607229,3.000000,250.860337,180.018506,76.152126,7.361735,1.851323,47.854458,17.822835,28.139566,40.429933
std,14703.567484,1.414282,1153.302321,811.551885,32.667135,39.031480,3.595536,231.620935,26.372115,135.910994,39.971153
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,45.000000,2.000000,3.000000,3.000000,69.105000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,104.000000,3.000000,22.000000,20.000000,89.500000,0.000000,0.000000,1.000000,4.000000,1.000000,39.530000
75%,200.000000,4.000000,145.000000,112.000000,100.000000,2.000000,2.815000,16.000000,26.990000,9.000000,75.610000
max,99999.000000,5.000000,15627.000000,11901.000000,100.000000,949.000000,67.000000,3450.000000,100.000000,1965.000000,100.000000


In [21]:
paed3_data.shape

(10375, 13)

In [22]:
paed3_data.to_csv (file_path + '/clean_paediatric_data.csv', index = False, header=True)